# 기본 설치

In [ ]:
from google.colab import drive
from os.path import join
ROOT = "/content/gdrive"
drive.mount(ROOT)
MY_GOOGLE_DRIVE_PATH = '/content/drive/Shareddrives/23-2 DataMining TeamProject'
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH) # 프로젝트 경로

Mounted at /content/gdrive


In [ ]:
!pip install selenium
!apt-get update
!pip install webdriver_manager
!pip install threadpoolctl==3.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [633 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://a

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service as ChromeService
import re
from pandas import DataFrame, Series
import os
import time

In [ ]:
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


# Crawling

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
ARTIST_FILE = 'artist.csv'
ARTIST_PATH = join(PROJECT_PATH, ARTIST_FILE)
temp_artist = pd.read_csv(ARTIST_FILE)

In [ ]:
artist = []

In [ ]:
temp_artist = np.array(temp_artist['Artist'])


for singer in temp_artist:
  singer = singer.lower()
  artist = np.append(artist,singer)


Music crawling

In [ ]:
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')

chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
url = 'http://www.melon.com'
driver.get(url)

html = driver.page_source   # 접속한 웹 페이지의 HTML 다운로드
soup = BeautifulSoup(html, 'html.parser')   # BeautifulSoup으로 읽음

In [ ]:
music_list = []

def crawlingByArtist(artist_name):
  driver.find_element(By.ID,'top_search').send_keys(artist_name)
  driver.find_element(By.CLASS_NAME,'btn_icon.search_m').click()
  driver.find_element(By.LINK_TEXT,"곡").click()

  music_more_info_list_count = []
  music_more_info_list_count = driver.find_elements(By.CLASS_NAME,"btn.btn_icon_detail")

  for i in range(0, len(music_more_info_list_count)):
    try:
      music_more_info_list = []
      music_more_info_list = driver.find_elements(By.CLASS_NAME,"btn.btn_icon_detail")
      music_more_info_list[i].click()
      music_info = {}
      song = driver.find_element(By.CLASS_NAME,"song_name").text
      artist = driver.find_element(By.CLASS_NAME,"artist").text
      album = driver.find_element(By.CSS_SELECTOR,"#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(2)").text
      released_date = driver.find_element(By.CSS_SELECTOR,"#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)").text
      genre = driver.find_element(By.CSS_SELECTOR,"#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)").text
      like = driver.find_element(By.CSS_SELECTOR,"#d_like_count").text

      try:
        driver.find_element(By.CLASS_NAME,'button_more.arrow_d').click()
        lyrics = driver.find_element(By.CLASS_NAME,"lyric.on").text

        music_info["song"] = song
        music_info["artist"] = artist
        music_info["album"] = album
        music_info["released_date"] = released_date
        music_info["genre"] = genre
        music_info["like"] = like
        music_info["lyrics"] = lyrics

        music_list.append(music_info)
        driver.back()
      except:
        print("가사가 없습니다", artist)
        driver.back()
    except:
      print("오류가 발생했습니다.")

In [ ]:
for j,singer in enumerate(artist):
  crawlingByArtist(singer)
  driver.back()
  driver.back()


# CSV 저장

In [ ]:
import csv

field_names = ['song','artist','album','released_date','genre','like','lyrics']

with open('music_lyrics.csv','w') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames= field_names)
  writer.writeheader()
  writer.writerows(music_list)